## Loss Functions

In [74]:
import torch
from torch.nn import MSELoss, BCELoss, BCEWithLogitsLoss, CrossEntropyLoss, NLLLoss
import torch.nn.functional as F

In [4]:
## RMSE LOSS
mse_loss = MSELoss()
y_hat = torch.randint(0,10,(10,), dtype=torch.float16)
y = torch.randint(0,10,(10,), dtype=torch.float16)
print(mse_loss(y_hat,y))
print(torch.mean((y_hat-y)**2))


tensor(8.8984, dtype=torch.float16)
tensor(8.8984, dtype=torch.float16)


In [67]:
## BCE LOSS [2 classes, predicting for ONE ]
bce_loss = BCELoss()
bce_logit_loss = BCEWithLogitsLoss()
batch_dim = 20
logits = torch.distributions.normal.Normal(0,3).sample((batch_dim,))
probs = F.sigmoid(logits)
target = torch.distributions.Bernoulli(0.5).sample((batch_dim,))

print(f"Manually computed loss: {-torch.mean(target * torch.log(probs) + (1-target) * torch.log(1-probs))}")
print(f"bce logit loss: {bce_logit_loss(logits, target)}")
print(f"bce  loss: {bce_loss(probs, target)}")



Manually computed loss: 1.5414930582046509
bce logit loss: 1.5414930582046509
bce  loss: 1.5414930582046509


In [127]:
## Multiclass 
from sklearn.preprocessing import LabelEncoder
import numpy as np

taget_label_encoder = LabelEncoder()
cross_entropy_loss = CrossEntropyLoss()
nll_loss = NLLLoss()


batch_dim = 100
output_class = 5
multiclass_logit = torch.distributions.normal.Normal(0,5).sample((batch_dim, output_class))
multiclass_probs = F.softmax(multiclass_logit, dim=-1)

multiclass_target_probs = torch.distributions.Uniform(0,1).sample((batch_dim, output_class))
multiclass_target_probs = multiclass_target_probs/torch.sum(multiclass_target_probs, dim=-1, keepdim=True)

manual_calculated_loss = torch.mean(-torch.sum(multiclass_target_probs * torch.log(multiclass_probs),dim=-1))
print(f"Manually calculated loss: {manual_calculated_loss}")

cross_entropy_loss(multiclass_probs, multiclass_target_probs)


Manually calculated loss: 6.016842842102051


tensor(1.6839)

In [121]:
manual_calculated_loss

tensor(15.3749)

In [111]:
multiclass_predicted_probs[0], multiclass_probs[0]

(tensor([0.8666, 0.0529, 0.6594, 0.3782, 0.0596]),
 tensor([0.3078, 0.0021, 0.5725, 0.0051, 0.1124]))

In [133]:

labels = np.random.choice(['apple', 'banana', 'guava', 'peer', 'orange'],batch_dim)
labels_encoded = taget_label_encoder.fit_transform(labels)
labels_tensor = torch.from_numpy(labels_encoded)


print(f"Cross Entropy Loss: {cross_entropy_loss(multiclass_logit, labels_tensor)}")
print(f"NLL Loss: {nll_loss(multiclass_probs, labels_tensor)}")


Cross Entropy Loss: 6.096521377563477
NLL Loss: -0.19184499979019165


In [141]:
labels_tensor.unsqueeze(-1).shape

torch.Size([100, 1])

In [148]:
multiclass_probs[0:5], labels_tensor[0:5]

(tensor([[1.5734e-05, 8.7318e-01, 1.9978e-04, 1.1427e-07, 1.2661e-01],
         [8.2024e-01, 4.2694e-03, 1.7478e-01, 7.1180e-04, 4.8543e-07],
         [2.2922e-01, 2.7264e-03, 2.1522e-02, 7.4254e-01, 3.9927e-03],
         [1.0611e-01, 1.5996e-01, 6.7737e-01, 4.2383e-02, 1.4183e-02],
         [1.5916e-06, 3.7337e-05, 6.2795e-07, 5.4157e-03, 9.9454e-01]]),
 tensor([0, 4, 2, 4, 3]))

In [157]:
print(torch.gather(multiclass_probs, dim=1, index=labels_tensor.unsqueeze(-1))[:5])
print(torch.gather(multiclass_probs, dim=0, index=labels_tensor.unsqueeze(-1))[:5])
multiclass_probs[:5]

tensor([[1.5734e-05],
        [4.8543e-07],
        [2.1522e-02],
        [1.4183e-02],
        [5.4157e-03]])
tensor([[1.5734e-05],
        [1.5916e-06],
        [2.2922e-01],
        [1.5916e-06],
        [1.0611e-01]])


tensor([[1.5734e-05, 8.7318e-01, 1.9978e-04, 1.1427e-07, 1.2661e-01],
        [8.2024e-01, 4.2694e-03, 1.7478e-01, 7.1180e-04, 4.8543e-07],
        [2.2922e-01, 2.7264e-03, 2.1522e-02, 7.4254e-01, 3.9927e-03],
        [1.0611e-01, 1.5996e-01, 6.7737e-01, 4.2383e-02, 1.4183e-02],
        [1.5916e-06, 3.7337e-05, 6.2795e-07, 5.4157e-03, 9.9454e-01]])

In [107]:
multiclass_probs[0][labels_tensor[0]]

tensor(0.0458)

In [131]:
input_tensor = torch.tensor([[10, 20, 30], 
                             [40, 50, 60]])

index_tensor = torch.tensor([[0, 2], 
                             [1, 1]])

output = torch.gather(input_tensor, dim=1, index=index_tensor)
print(output)

tensor([[10, 30],
        [50, 50]])


In [129]:
input_tensor.shape

torch.Size([2, 3])

In [132]:
index_tensor = torch.tensor([[0, 0, 1], 
                             [1, 0, 0]])

output = torch.gather(input_tensor, dim=0, index=index_tensor)
print(output)

tensor([[10, 20, 60],
        [40, 20, 30]])


In [166]:
a = torch.randint(0,20, (2,3))
a

tensor([[19, 11, 13],
        [ 3,  8,  5]])

In [ ]:
torch.gather(a, dim=0, index= torch.tensor([[2],[1]]))

tensor([[18],
        [10]])

In [174]:
torch.gather(a, dim=1, index=torch.tensor([[0],[0]]))

tensor([[19],
        [ 3]])

In [179]:
print(a)
torch.gather(a, dim=0, index=torch.tensor([[0],[0],[0]]))

tensor([[19, 11, 13],
        [ 3,  8,  5]])


tensor([[19],
        [19],
        [19]])

In [183]:
a

tensor([[19, 11, 13],
        [ 3,  8,  5]])

In [189]:
torch.gather(a, dim=0, index=torch.tensor([[0,0],[1,1]]))

tensor([[19, 11],
        [ 3,  8]])

In [191]:
input_tensor = torch.randint(0,20, (3,4))
print(input_tensor.shape)
input_tensor

torch.Size([3, 4])


tensor([[19, 11,  7, 11],
        [ 7,  0,  6,  9],
        [10,  9,  5,  4]])

In [ ]:

index_tensor_along_dim_0

tensor([[2, 1, 0, 1],
        [0, 0, 0, 0]])

In [208]:
torch.gather(input_tensor, dim=0, index=index_tensor_along_dim_0)

tensor([[10,  0,  7,  9],
        [19, 11,  7, 11]])

In [210]:
index_tensor_along_dim_1 = torch.tensor([[0,2,1,0,0]])

In [211]:
torch.gather(input_tensor, dim=1, index=index_tensor_along_dim_1)

tensor([[19,  7, 11, 19, 19]])

In [219]:
index_tensor_along_dim_0 = torch.tensor([[0,1],[1,2],[2,0],[0,1]])
torch.gather(input_tensor, dim=0, index=index_tensor_along_dim_0)

tensor([[19,  0],
        [ 7,  9],
        [10, 11],
        [19,  0]])

In [217]:
input_tensor

tensor([[19, 11,  7, 11],
        [ 7,  0,  6,  9],
        [10,  9,  5,  4]])